In [3]:
pip install pandas numpy faker

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
from faker import Faker
import random

In [10]:
fake = Faker()
np.random.seed(42)
random.seed(42)

N_CUSTOMERS = 3000

customers = []

for i in range(1, N_CUSTOMERS + 1):
    age = np.random.randint(21, 65)
    income = np.random.choice(
        [200000, 350000, 500000, 800000, 1200000, 1800000],
        p=[0.15, 0.25, 0.25, 0.2, 0.1, 0.05]
    )
    customers.append({
        "customer_id": i,
        "age": age,
        "gender": random.choice(["Male", "Female"]),
        "income": income,
        "occupation": random.choice(["Salaried", "Self-Employed", "Student", "Retired"]),
        "city": fake.city(),
        "tenure_years": np.random.randint(1, 20)
    })

customers_df = pd.DataFrame(customers)

In [11]:
accounts = []

for i in range(1, N_CUSTOMERS + 1):
    balance = np.random.normal(80000, 60000)
    balance = max(2000, balance)

    accounts.append({
        "customer_id": i,
        "avg_balance": round(balance, 2),
        "monthly_deposits": np.random.randint(1, 15),
        "monthly_withdrawals": np.random.randint(1, 20),
        "digital_txn_ratio": round(np.random.uniform(0.2, 1.0), 2)
    })

accounts_df = pd.DataFrame(accounts)

In [12]:
cards = []

for i in range(1, N_CUSTOMERS + 1):
    has_card = np.random.rand() < 0.6

    if has_card:
        limit = np.random.choice([50000, 100000, 200000, 300000])
        utilization = round(np.random.uniform(0.1, 0.95), 2)
        cards.append({
            "customer_id": i,
            "card_limit": limit,
            "utilization_ratio": utilization,
            "delinquency_12m": np.random.choice([0, 1], p=[0.9, 0.1])
        })

cards_df = pd.DataFrame(cards)

In [13]:
loans = []

for i in range(1, N_CUSTOMERS + 1):
    has_loan = np.random.rand() < 0.5

    if has_loan:
        loan_type = random.choice(["Personal", "Auto", "Home"])
        emi = np.random.randint(3000, 40000)
        overdue = np.random.choice([0, 1], p=[0.88, 0.12])

        loans.append({
            "customer_id": i,
            "loan_type": loan_type,
            "monthly_emi": emi,
            "overdue_flag": overdue
        })

loans_df = pd.DataFrame(loans)

In [14]:
investments = []

for i in range(1, N_CUSTOMERS + 1):
    invest = np.random.rand() < 0.4

    if invest:
        investments.append({
            "customer_id": i,
            "mf_investment": np.random.randint(10000, 300000),
            "fd_amount": np.random.choice([0, 50000, 100000, 200000]),
            "insurance_policy": np.random.choice([0, 1], p=[0.4, 0.6])
        })

investments_df = pd.DataFrame(investments)

In [15]:
transactions = []

for i in range(1, N_CUSTOMERS + 1):
    for m in range(1, 7):  # last 6 months
        transactions.append({
            "customer_id": i,
            "month": m,
            "shopping_spend": np.random.randint(1000, 15000),
            "travel_spend": np.random.randint(0, 12000),
            "utility_spend": np.random.randint(500, 5000),
            "wallet_spend": np.random.randint(0, 8000)
        })

transactions_df = pd.DataFrame(transactions)

In [16]:
customers_df.to_csv("customers.csv", index=False)
accounts_df.to_csv("accounts.csv", index=False)
cards_df.to_csv("credit_cards.csv", index=False)
loans_df.to_csv("loans.csv", index=False)
investments_df.to_csv("investments.csv", index=False)
transactions_df.to_csv("transactions.csv", index=False)

print("All BFSI datasets generated successfully.")

All BFSI datasets generated successfully.


In [17]:
customers = pd.read_csv("C:/Users/pktal/customers.csv")
accounts = pd.read_csv("C:/Users/pktal/accounts.csv")
cards = pd.read_csv("C:/Users/pktal/credit_cards.csv")
loans = pd.read_csv("C:/Users/pktal/loans.csv")
investments = pd.read_csv("C:/Users/pktal/investments.csv")
transactions = pd.read_csv("C:/Users/pktal/transactions.csv")

In [18]:
print(customers.shape)
print(accounts.shape)
print(cards.shape)
print(loans.shape)
print(investments.shape)
print(transactions.shape)

(3000, 7)
(3000, 5)
(1781, 4)
(1465, 4)
(1238, 4)
(18000, 6)


In [19]:
customers.head()

,customer_id,age,gender,income,occupation,city,tenure_years
0,1,59,Male,800000,Salaried,Danieltown,15
1,2,63,Female,800000,Self-Employed,Littleberg,7
2,3,39,Male,200000,Self-Employed,Markview,11
3,4,44,Male,350000,Salaried,Lake Robertstad,8
4,5,44,Female,800000,Salaried,Delgadoburgh,2


In [23]:
investments.isna().sum()

customer_id         0
mf_investment       0
fd_amount           0
insurance_policy    0
dtype: int64

In [24]:
customers["gender"] = customers["gender"].str.capitalize()
customers["occupation"] = customers["occupation"].str.title()

In [25]:
accounts["avg_balance"] = accounts["avg_balance"].clip(lower=0)
accounts.fillna(0, inplace=True)

In [26]:
cards.fillna(0, inplace=True)


In [27]:
loans.fillna(0, inplace=True)


In [28]:
investments.fillna(0, inplace=True)


In [29]:
txn_agg = transactions.groupby("customer_id").agg({
    "shopping_spend": "sum",
    "travel_spend": "sum",
    "utility_spend": "sum",
    "wallet_spend": "sum"
}).reset_index()

txn_agg["total_spend_6m"] = txn_agg[
    ["shopping_spend","travel_spend","utility_spend","wallet_spend"]
].sum(axis=1)

In [30]:
cust_360 = customers.merge(accounts, on="customer_id", how="left")
cust_360 = cust_360.merge(cards, on="customer_id", how="left")
cust_360 = cust_360.merge(loans, on="customer_id", how="left")
cust_360 = cust_360.merge(investments, on="customer_id", how="left")
cust_360 = cust_360.merge(txn_agg, on="customer_id", how="left")

In [31]:
product_cols = [
    "card_limit","utilization_ratio","delinquency_12m",
    "monthly_emi","overdue_flag",
    "mf_investment","fd_amount","insurance_policy"
]

cust_360[product_cols] = cust_360[product_cols].fillna(0)
cust_360.fillna(0, inplace=True)

In [32]:
cust_360["has_credit_card"] = np.where(cust_360["card_limit"] > 0, 1, 0)
cust_360["has_loan"] = np.where(cust_360["monthly_emi"] > 0, 1, 0)
cust_360["has_investments"] = np.where(
    (cust_360["mf_investment"] > 0) | (cust_360["fd_amount"] > 0), 1, 0
)

In [33]:
cust_360.to_csv("C:/Users/pktal/merged_customer_360.csv", index=False)
print("Customer 360 dataset saved successfully.")

Customer 360 dataset saved successfully.


In [34]:
cust_360.describe()

,customer_id,age,income,tenure_years,avg_balance,monthly_deposits,monthly_withdrawals,digital_txn_ratio,card_limit,utilization_ratio,...,fd_amount,insurance_policy,shopping_spend,travel_spend,utility_spend,wallet_spend,total_spend_6m,has_credit_card,has_loan,has_investments
count,3000.000000,3000.000000,3.000000e+03,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,...,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,1500.500000,42.690000,6.083000e+05,10.122333,82001.446217,7.446667,10.299000,0.600020,94883.333333,0.315087,...,36116.666667,0.247667,47811.838000,36033.620333,16522.862667,24020.282333,124388.603333,0.593667,0.488333,0.412667
std,866.169729,12.556424,3.930491e+05,5.570133,54998.916944,4.091300,5.433732,0.233104,106919.581873,0.323146,...,64000.925215,0.431729,9965.903784,8500.159812,3097.964026,5681.530088,14635.432952,0.491230,0.499947,0.492396
min,1.000000,21.000000,2.000000e+05,1.000000,2000.000000,1.000000,1.000000,0.200000,0.000000,0.000000,...,0.000000,0.000000,13735.000000,8695.000000,5329.000000,6911.000000,74402.000000,0.000000,0.000000,0.000000
25%,750.750000,32.000000,3.500000e+05,5.000000,38256.742500,4.000000,6.000000,0.400000,0.000000,0.000000,...,0.000000,0.000000,40935.750000,30156.750000,14341.750000,20096.000000,114549.000000,0.000000,0.000000,0.000000
50%,1500.500000,43.000000,5.000000e+05,10.000000,80392.975000,7.000000,10.000000,0.600000,50000.000000,0.240000,...,0.000000,0.000000,47837.500000,36004.000000,16601.500000,24025.500000,124558.500000,1.000000,0.000000,0.000000
75%,2250.250000,53.000000,8.000000e+05,15.000000,119842.545000,11.000000,15.000000,0.810000,200000.000000,0.600000,...,50000.000000,0.000000,54761.500000,42075.750000,18668.250000,28003.500000,134460.250000,1.000000,1.000000,1.000000
max,3000.000000,64.000000,1.800000e+06,19.000000,266544.650000,14.000000,19.000000,1.000000,300000.000000,0.950000,...,200000.000000,1.000000,79514.000000,63001.000000,26888.000000,41056.000000,177293.000000,1.000000,1.000000,1.000000


In [35]:
cust_360[["has_credit_card","has_loan","has_investments"]].mean()

has_credit_card    0.593667
has_loan           0.488333
has_investments    0.412667
dtype: float64